In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [2]:
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

# Clustering

In [3]:
features = pd.read_csv('datasets/item_features.csv')

In [4]:
features

,item_id,feature_category_id,feature_value_id
0,2,56,365
1,2,62,801
2,2,68,351
3,2,33,802
4,2,72,75
...,...,...,...
471746,28143,68,351
471747,28143,55,390
471748,28143,11,109
471749,28143,73,91


In [5]:
n_pca_comp = 16
number_of_clusters = 4

In [6]:
cluster_assign = pd.read_csv('datasets/cluster_assign.csv')
cluster_assign = cluster_assign[['item_id2', 'Cluster_pred']]
cluster_assign

,item_id2,Cluster_pred
0,2,2
1,3,1
2,4,1
3,7,0
4,8,1
...,...,...
23686,28139,1
23687,28140,3
23688,28141,1
23689,28142,0


# Feature Engineering

- Primer producto visto
- Instante en el que se ha visto el primer producto
- Último producto visto
- Número total de productos vistos
- Instante en el que se ha visto el último producto
- Duración total de la sesión
- Tiempo medio dedicado a cada producto
- Período del día en el cual la sesión comenzó (madrugada, día, tarde o noche)
- Categoria más vista en la sesión
- Número de categorias distintas vistas en la sesión
- Duración de la sesión

In [7]:
df_sessions = pd.read_csv('datasets/train_sessions.csv')

In [8]:
df_sessions.head()

,session_id,item_id,date
0,3,9655,2020-12-18 21:25:00.373
1,3,9655,2020-12-18 21:19:48.093
2,13,15654,2020-03-13 19:35:27.136
3,18,18316,2020-08-26 19:18:30.833
4,18,2507,2020-08-26 19:16:31.211


In [9]:
session_sorted = df_sessions.sort_values(by=['date'])

# Instante en el que se ha visto el primer producto
begin_df = session_sorted.groupby('session_id')['date'].first().rename('start_date')

# Primer producto visto
first_prod = session_sorted.groupby('session_id')['item_id'].first().rename('first_prod')

# Número total de productos vistos
prod_count = session_sorted.groupby('session_id')['item_id'].count().rename('prod_count')

# Instante en el que se ha visto el último producto
end_df = session_sorted.groupby('session_id')['date'].last().rename('end_date')

# Último producto visto
last_prod = session_sorted.groupby('session_id')['item_id'].last().rename('last_prod')

# Categoria más vista en la sesión
join_cat = session_sorted.merge(features, on='item_id', how='left').sort_values(by=['feature_category_id'])
common_cat = join_cat.groupby(['session_id'])['feature_category_id'].value_counts().rename('most_common_cat')
common_cat_df = common_cat.to_frame(name = None).reset_index()
common_cat = common_cat_df.groupby(['session_id'])['feature_category_id'].first().rename('most_common_cat')

# Número de categorias distintas vistas en la sesión
count_num_unique_cat = common_cat_df.groupby(['session_id'])['feature_category_id'].count().rename('count_num_unique_cat')


# Concatenamos los dataframes
times_df = pd.concat([begin_df, end_df, first_prod, last_prod, 
                      prod_count, common_cat, count_num_unique_cat
                     ],axis=1)

# Modificamos el formato de fehca
times_df['start_date'] = pd.to_datetime(times_df['start_date'])
times_df['end_date'] = pd.to_datetime(times_df['end_date'])

# Duración de la sesión
times_df['time_diff'] = (times_df.end_date-times_df.start_date).astype('timedelta64[s]')
times_df['time_diff'] = times_df['time_diff'].astype(int)

# Tiempo medio dedicado a cada producto
times_df['time_per_prod'] = (times_df['time_diff']/times_df['prod_count']).astype(int)

# Momento del día en que se inició la sesión
mask=(times_df.start_date.dt.hour>=0) & (times_df.start_date.dt.hour<7)
times_df.loc[mask,'time_first_prod'] = 'madrugada'
mask=(times_df.start_date.dt.hour>=7) & (times_df.start_date.dt.hour<12)
times_df.loc[mask,'time_first_prod'] = 'día'
mask=(times_df.start_date.dt.hour>=12) & (times_df.start_date.dt.hour<18)
times_df.loc[mask,'time_first_prod'] = 'tarde'
mask=(times_df.start_date.dt.hour>=18) & (times_df.start_date.dt.hour<24)
times_df.loc[mask,'time_first_prod'] = 'noche'


In [10]:
sessions=times_df
sessions

,start_date,end_date,first_prod,last_prod,prod_count,most_common_cat,count_num_unique_cat,time_diff,time_per_prod,time_first_prod
session_id,,,,,,,,,,
3,2020-12-18 21:19:48.093,2020-12-18 21:25:00.373,9655,9655,2,2,18,312,156,noche
13,2020-03-13 19:35:27.136,2020-03-13 19:35:27.136,15654,15654,1,4,20,0,0,noche
18,2020-08-26 19:15:47.232,2020-08-26 19:18:30.833,4026,18316,3,1,25,163,54,noche
19,2020-11-02 16:30:36.378,2020-11-02 16:48:39.343,19896,8268,17,4,28,1082,63,tarde
24,2020-02-26 17:22:48.903,2020-02-26 18:24:32.770,2927,18476,9,30,43,3703,411,tarde
...,...,...,...,...,...,...,...,...,...,...
4439986,2021-05-13 11:46:56.606,2021-05-13 11:55:50.151,23502,27733,6,4,18,533,88,día
4439990,2020-08-22 11:38:48.785,2020-08-22 12:36:27.326,22093,22551,11,4,46,3458,314,día
4439994,2020-11-27 20:08:37.262,2020-11-27 20:08:37.262,25357,25357,1,3,25,0,0,noche


# Productos comprados (Purchases)

In [11]:
purchases = pd.read_csv('datasets/train_purchases.csv').set_index('session_id')
purchases['date']=pd.to_datetime(purchases['date'])
purchases.rename(columns={'item_id':'item_purch','date':'purch_date'}, inplace=True)
purchases.head()

,item_purch,purch_date
session_id,,
3,15085,2020-12-18 21:26:47.986
13,18626,2020-03-13 19:36:15.507
18,24911,2020-08-26 19:20:32.049
19,12534,2020-11-02 17:16:45.920
24,13226,2020-02-26 18:27:44.114


In [12]:
purchases.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 3 to 4440001
Data columns (total 2 columns):
 #   Column      Non-Null Count    Dtype         
---  ------      --------------    -----         
 0   item_purch  1000000 non-null  int64         
 1   purch_date  1000000 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1)
memory usage: 22.9 MB


# Features
- Tabla con sesiones y veces que se ha visto cada clúster

In [13]:
item_clust = cluster_assign[['item_id2', 'Cluster_pred']].set_index('item_id2')
item_clust

,Cluster_pred
item_id2,
2,2
3,1
4,1
7,0
8,1
...,...
28139,1
28140,3
28141,1


In [14]:
df_sess_clust=df_sessions.merge(item_clust, left_on='item_id', right_index=True)
df_sess_clust

,session_id,item_id,date,Cluster_pred
0,3,9655,2020-12-18 21:25:00.373,3
1,3,9655,2020-12-18 21:19:48.093,3
103605,97085,9655,2020-12-27 20:22:13.011,3
162423,152155,9655,2020-11-19 16:26:18.468,3
225780,210806,9655,2021-01-02 05:37:37.183,3
...,...,...,...,...
4737630,4434058,3416,2020-11-14 23:33:46.655,1
4737631,4434058,15964,2020-11-14 23:33:58.652,1
4739963,4436170,27863,2020-07-20 06:36:30.611,0
4741053,4437232,7735,2020-01-02 20:07:06.367,1


In [15]:
df_clust_count = df_sess_clust.groupby(['session_id', 'Cluster_pred'])['session_id'].aggregate('count').unstack().fillna(0)


In [16]:
df_clust_count

Cluster_pred,0,1,2,3
session_id,,,,
3,0.0,0.0,0.0,2.0
13,0.0,0.0,1.0,0.0
18,0.0,3.0,0.0,0.0
19,0.0,17.0,0.0,0.0
24,3.0,0.0,5.0,1.0
...,...,...,...,...
4439986,0.0,0.0,6.0,0.0
4439990,0.0,8.0,2.0,1.0
4439994,0.0,1.0,0.0,0.0


Cuál es el cluster más visto de cada sesión?

In [17]:
df_clust_count["most_seen_cluster"] = df_clust_count.idxmax(axis=1)
df_clust_count

Cluster_pred,0,1,2,3,most_seen_cluster
session_id,,,,,
3,0.0,0.0,0.0,2.0,3
13,0.0,0.0,1.0,0.0,2
18,0.0,3.0,0.0,0.0,1
19,0.0,17.0,0.0,0.0,1
24,3.0,0.0,5.0,1.0,2
...,...,...,...,...,...
4439986,0.0,0.0,6.0,0.0,2
4439990,0.0,8.0,2.0,1.0,1
4439994,0.0,1.0,0.0,0.0,1


# Concatenar todos los dataframes

In [18]:
df=pd.concat([sessions, purchases, df_clust_count], axis=1)
df

,start_date,end_date,first_prod,last_prod,prod_count,most_common_cat,count_num_unique_cat,time_diff,time_per_prod,time_first_prod,item_purch,purch_date,0,1,2,3,most_seen_cluster
session_id,,,,,,,,,,,,,,,,,
3,2020-12-18 21:19:48.093,2020-12-18 21:25:00.373,9655,9655,2,2,18,312,156,noche,15085,2020-12-18 21:26:47.986,0.0,0.0,0.0,2.0,3
13,2020-03-13 19:35:27.136,2020-03-13 19:35:27.136,15654,15654,1,4,20,0,0,noche,18626,2020-03-13 19:36:15.507,0.0,0.0,1.0,0.0,2
18,2020-08-26 19:15:47.232,2020-08-26 19:18:30.833,4026,18316,3,1,25,163,54,noche,24911,2020-08-26 19:20:32.049,0.0,3.0,0.0,0.0,1
19,2020-11-02 16:30:36.378,2020-11-02 16:48:39.343,19896,8268,17,4,28,1082,63,tarde,12534,2020-11-02 17:16:45.920,0.0,17.0,0.0,0.0,1
24,2020-02-26 17:22:48.903,2020-02-26 18:24:32.770,2927,18476,9,30,43,3703,411,tarde,13226,2020-02-26 18:27:44.114,3.0,0.0,5.0,1.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4439986,2021-05-13 11:46:56.606,2021-05-13 11:55:50.151,23502,27733,6,4,18,533,88,día,2915,2021-05-13 11:56:37.464,0.0,0.0,6.0,0.0,2
4439990,2020-08-22 11:38:48.785,2020-08-22 12:36:27.326,22093,22551,11,4,46,3458,314,día,8786,2020-08-22 14:28:22.382,0.0,8.0,2.0,1.0,1
4439994,2020-11-27 20:08:37.262,2020-11-27 20:08:37.262,25357,25357,1,3,25,0,0,noche,21630,2020-11-27 20:10:28.961,0.0,1.0,0.0,0.0,1


Cuál era el cluster del primer y del último producto visto?

In [19]:
df = df.reset_index().merge(item_clust, how='left', left_on='first_prod', right_on=item_clust.index).set_index('session_id')
df = df.rename(columns = {'Cluster_pred':'first_item_cluster'})
df = df.reset_index().merge(item_clust, how='left', left_on='last_prod', right_on=item_clust.index).set_index('session_id')
df = df.rename(columns = {'Cluster_pred':'last_item_cluster'})

In [20]:
df

,start_date,end_date,first_prod,last_prod,prod_count,most_common_cat,count_num_unique_cat,time_diff,time_per_prod,time_first_prod,item_purch,purch_date,0,1,2,3,most_seen_cluster,first_item_cluster,last_item_cluster
session_id,,,,,,,,,,,,,,,,,,,
3,2020-12-18 21:19:48.093,2020-12-18 21:25:00.373,9655,9655,2,2,18,312,156,noche,15085,2020-12-18 21:26:47.986,0.0,0.0,0.0,2.0,3,3,3
13,2020-03-13 19:35:27.136,2020-03-13 19:35:27.136,15654,15654,1,4,20,0,0,noche,18626,2020-03-13 19:36:15.507,0.0,0.0,1.0,0.0,2,2,2
18,2020-08-26 19:15:47.232,2020-08-26 19:18:30.833,4026,18316,3,1,25,163,54,noche,24911,2020-08-26 19:20:32.049,0.0,3.0,0.0,0.0,1,1,1
19,2020-11-02 16:30:36.378,2020-11-02 16:48:39.343,19896,8268,17,4,28,1082,63,tarde,12534,2020-11-02 17:16:45.920,0.0,17.0,0.0,0.0,1,1,1
24,2020-02-26 17:22:48.903,2020-02-26 18:24:32.770,2927,18476,9,30,43,3703,411,tarde,13226,2020-02-26 18:27:44.114,3.0,0.0,5.0,1.0,2,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4439986,2021-05-13 11:46:56.606,2021-05-13 11:55:50.151,23502,27733,6,4,18,533,88,día,2915,2021-05-13 11:56:37.464,0.0,0.0,6.0,0.0,2,2,2
4439990,2020-08-22 11:38:48.785,2020-08-22 12:36:27.326,22093,22551,11,4,46,3458,314,día,8786,2020-08-22 14:28:22.382,0.0,8.0,2.0,1.0,1,1,1
4439994,2020-11-27 20:08:37.262,2020-11-27 20:08:37.262,25357,25357,1,3,25,0,0,noche,21630,2020-11-27 20:10:28.961,0.0,1.0,0.0,0.0,1,1,1


In order to reduce the training time, we will need to reduce the number of sessions to 112055 (id of the session must be less or equal than 500000).

In [21]:
df_limited = df[df.index <= 50000]
#df_limited = df_limited[df_limited.prod_count >= 2]
df_limited.to_csv('datasets/dataset_merge.csv')
df_limited

,start_date,end_date,first_prod,last_prod,prod_count,most_common_cat,count_num_unique_cat,time_diff,time_per_prod,time_first_prod,item_purch,purch_date,0,1,2,3,most_seen_cluster,first_item_cluster,last_item_cluster
session_id,,,,,,,,,,,,,,,,,,,
3,2020-12-18 21:19:48.093,2020-12-18 21:25:00.373,9655,9655,2,2,18,312,156,noche,15085,2020-12-18 21:26:47.986,0.0,0.0,0.0,2.0,3,3,3
13,2020-03-13 19:35:27.136,2020-03-13 19:35:27.136,15654,15654,1,4,20,0,0,noche,18626,2020-03-13 19:36:15.507,0.0,0.0,1.0,0.0,2,2,2
18,2020-08-26 19:15:47.232,2020-08-26 19:18:30.833,4026,18316,3,1,25,163,54,noche,24911,2020-08-26 19:20:32.049,0.0,3.0,0.0,0.0,1,1,1
19,2020-11-02 16:30:36.378,2020-11-02 16:48:39.343,19896,8268,17,4,28,1082,63,tarde,12534,2020-11-02 17:16:45.920,0.0,17.0,0.0,0.0,1,1,1
24,2020-02-26 17:22:48.903,2020-02-26 18:24:32.770,2927,18476,9,30,43,3703,411,tarde,13226,2020-02-26 18:27:44.114,3.0,0.0,5.0,1.0,2,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49976,2021-01-03 15:10:27.864,2021-01-03 15:27:03.675,8114,23287,2,3,28,995,497,tarde,24900,2021-01-03 15:28:38.196,0.0,2.0,0.0,0.0,1,1,1
49980,2021-03-31 20:09:31.139,2021-03-31 20:12:58.553,23034,2881,3,4,37,207,69,noche,3057,2021-03-31 20:14:19.965,0.0,2.0,1.0,0.0,1,1,1
49991,2020-08-29 16:59:45.686,2020-08-29 16:59:45.686,7490,7490,1,4,19,0,0,tarde,7590,2020-08-29 17:01:09.711,0.0,0.0,1.0,0.0,2,2,2
